# Imports

In [ ]:
import pandas as pd 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import regex as re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime as dt
#!pip install category_encoders
import category_encoders as ce
#!pip install pyspark
import pyspark as ps
from pyspark.sql.types import IntegerType
from pyspark import SparkContext,SparkFiles
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import builtins
round = getattr(builtins, "round")
abs = getattr(builtins, "abs")
import functools
from csv import reader
import copy

     |████████████████████████████████| 80 kB 2.0 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 55.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9f4fdd6f3b9a239bdaf4c802a45e3aeb48b2062f2ade9f43c1d7a6875cc39526
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Kaggle API

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [ ]:
!kaggle competitions download -c football-match-probability-prediction

 99% 64.0M/64.9M [00:00<00:00, 226MB/s]
100% 64.9M/64.9M [00:00<00:00, 218MB/s]


In [ ]:
!unzip football-match-probability-prediction

Archive:  football-match-probability-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_target_and_scores.csv  


# Spark Session

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Football Match Predictions") \
    .getOrCreate()
sc =SparkContext.getOrCreate()

# Preprocessing

## Reading the Data

In [ ]:
matches_df = pd.read_csv("train.csv", low_memory=False)
matches_spark_df = spark.read.csv('train.csv',header=True) 
matches_test_df = pd.read_csv("test.csv")
matches_test_spark_df = spark.read.csv('test.csv',header=True) 
avg_values={}
avg_columns = {}
diff_columns={}
header=[]

## Historical Columns

1-home_team_history_match_date

In [ ]:
home_team_history_match_date = list()
for i in range(1,11):
  home_team_history_match_date.append("home_team_history_match_date_%d"%i)
#matches_df[home_team_history_match_date]  

2- home_team_history_is_play_home

In [ ]:
home_team_history_is_play_home = list()
for i in range(1,11):
  home_team_history_is_play_home.append("home_team_history_is_play_home_%d"%i)
#matches_df[home_team_history_is_play_home]  

3-home_team_history_is_cup

In [ ]:
home_team_history_is_cup = list()
for i in range(1,11):
  home_team_history_is_cup.append("home_team_history_is_cup_%d"%i)
#matches_df[home_team_history_is_cup]  

4-home_team_history_goal

In [ ]:
home_team_history_goal = list()
for i in range(1,11):
  home_team_history_goal.append("home_team_history_goal_%d"%i)
#matches_df[home_team_history_goal]  

5-home_team_history_opponent_goal

In [ ]:
home_team_history_opponent_goal = list()
for i in range(1,11):
  home_team_history_opponent_goal.append("home_team_history_opponent_goal_%d"%i)
#matches_df[home_team_history_opponent_goal]  

6-home_team_history_rating

In [ ]:
home_team_history_rating = list()
for i in range(1,11):
  home_team_history_rating.append("home_team_history_rating_%d"%i)
#matches_df[home_team_history_rating]  

7-home_team_history_opponent_rating

In [ ]:
home_team_history_opponent_rating = list()
for i in range(1,11):
  home_team_history_opponent_rating.append("home_team_history_opponent_rating_%d"%i)
#matches_df[home_team_history_opponent_rating]  

8-home_team_history_coach

In [ ]:
home_team_history_coach = list()
for i in range(1,11):
  home_team_history_coach.append("home_team_history_coach_%d"%i)
#matches_df[home_team_history_coach]  

9-home_team_history_league_id

In [ ]:
home_team_history_league_id = list()
for i in range(1,11):
  home_team_history_league_id.append("home_team_history_league_id_%d"%i)
#matches_df[home_team_history_league_id]  

10-away_team_history_match_date

In [ ]:
away_team_history_match_date = list()
for i in range(1,11):
  away_team_history_match_date.append("away_team_history_match_date_%d"%i)
#matches_df[away_team_history_match_date]  

11-away_team_history_is_play_home

In [ ]:
away_team_history_is_play_home = list()
for i in range(1,11):
  away_team_history_is_play_home.append("away_team_history_is_play_home_%d"%i)
#matches_df[away_team_history_is_play_home]  

12-away_team_history_is_cup

In [ ]:
away_team_history_is_cup = list()
for i in range(1,11):
  away_team_history_is_cup.append("away_team_history_is_cup_%d"%i)
#matches_df[away_team_history_is_cup]  

13-away_team_history_goal

In [ ]:
away_team_history_goal = list()
for i in range(1,11):
  away_team_history_goal.append("away_team_history_goal_%d"%i)
#matches_df[away_team_history_goal]  

14-away_team_history_opponent_goal

In [ ]:
away_team_history_opponent_goal = list()
for i in range(1,11):
  away_team_history_opponent_goal.append("away_team_history_opponent_goal_%d"%i)
#matches_df[away_team_history_opponent_goal]  

15-away_team_history_rating

In [ ]:
away_team_history_rating = list()
for i in range(1,11):
  away_team_history_rating.append("away_team_history_rating_%d"%i)
#matches_df[away_team_history_rating]  

16-away_team_history_opponent_rating

In [ ]:
away_team_history_opponent_rating = list()
for i in range(1,11):
  away_team_history_opponent_rating.append("away_team_history_opponent_rating_%d"%i)
#matches_df[away_team_history_opponent_rating]  

17-away_team_history_coach

In [ ]:
away_team_history_coach = list()
for i in range(1,11):
  away_team_history_coach.append("away_team_history_coach_%d"%i)
#matches_df[away_team_history_coach]  

18-away_team_history_league_id

In [ ]:
away_team_history_league_id = list()
for i in range(1,11):
  away_team_history_league_id.append("away_team_history_league_id_%d"%i)
#matches_df[away_team_history_league_id]  

## NAN Values

In [ ]:
def remove_NAN(input_df, train = True,mode="MaxAvg"):

  output_df = input_df
  if mode=="MaxAvg":
    columns = list()
    columns_hist_goals=list()
    columns_hist_ratings=list()
    columns_match_date=list()
    columns_match_date=['match_date']
    columns = ['home_team_name', 'away_team_name',  'league_name', 'league_id', 'home_team_coach_id', 'away_team_coach_id', 'is_cup']
    if train == True:
      columns.extend(['target'])   

    # History Match Dates
    columns_match_date.extend(home_team_history_match_date)
    columns_match_date.extend(away_team_history_match_date)

    # History is Play Home
    columns.extend(home_team_history_is_play_home)
    columns.extend(away_team_history_is_play_home)
    # History is cup
    columns.extend(home_team_history_is_cup)
    columns.extend(away_team_history_is_cup)

    # History Goal
    columns_hist_goals.extend(home_team_history_goal)
    columns_hist_goals.extend(away_team_history_goal)
    # History Opponent Goal
    columns_hist_goals.extend(home_team_history_opponent_goal)
    columns_hist_goals.extend(away_team_history_opponent_goal)

    # History Rating
    columns_hist_ratings.extend(home_team_history_rating)
    columns_hist_ratings.extend(away_team_history_rating)
    # History Opponent Rating
    columns_hist_ratings.extend(home_team_history_opponent_rating)
    columns_hist_ratings.extend(away_team_history_opponent_rating)

    # History Coach
    columns.extend(home_team_history_coach)
    columns.extend(away_team_history_coach)
    # History League ID
    columns.extend(home_team_history_league_id)
    columns.extend(away_team_history_league_id)

    if train == True:
      for col in columns_hist_goals:
          avgC=output_df.select(avg(col)).collect()
          avg_values[col]=str(round(avgC[0][0]))
          output_df=output_df.fillna(str(round(avgC[0][0])),subset=col)
          
      for col in columns_hist_ratings:    
          avgC=output_df.select(avg(col)).collect()
          avg_values[col]=str(avgC[0][0])
          output_df=output_df.fillna(str(avgC[0][0]),subset=col)
          
      for col in columns_match_date:
          avgD=output_df.select(avg(to_timestamp(col)).cast("timestamp")).collect()
          avgDnew=str(avgD[0][0]).split('.')
          avg_values[col]=avgDnew[0]
          output_df=output_df.fillna(avgDnew[0],subset=col)
        
      for col in columns:
          mode=output_df.groupby(col).count().orderBy("count", ascending=False).head(2)
          if str(mode[0][0])=="None":
            avg_values[col]=str(mode[1][0])
            output_df=output_df.fillna(str(mode[1][0]),subset=col)
          else:  
            avg_values[col]=str(mode[0][0])
            output_df=output_df.fillna(str(mode[0][0]),subset=col)
    else:
      for col in columns_hist_goals:
        avgC=avg_values[col]
        output_df=output_df.fillna(avgC,subset=col)
      
      for col in columns_hist_ratings:    
        avgC=avg_values[col]
        output_df=output_df.fillna(avgC,subset=col)
      
      for col in columns_match_date:
        avgD=avg_values[col]
        output_df=output_df.fillna(avgD,subset=col)
    
      for col in columns:
        mode=avg_values[col]
        output_df=output_df.fillna(mode,subset=col)

  elif mode=="0":
    output_df=output_df.fillna(0)
  else:
    output_df=output_df.dropna(how="any")

  return output_df


## Removing Outliers

In [ ]:
def remove_outliers(input_df):
  output_df = input_df.copy()
  return output_df

In [ ]:
def remove_outliers_spark(input_df,mode="remove"):
  
  columns=list()
  columns2=list()
  # History Rating
  columns.extend(home_team_history_rating)
  columns.extend(away_team_history_rating)
  # History Opponent Rating
  columns.extend(home_team_history_opponent_rating)
  columns.extend(away_team_history_opponent_rating)

  # History Goal
  columns2.extend(home_team_history_goal)
  columns2.extend(away_team_history_goal)

  # History Opponent Goal
  columns2.extend(home_team_history_opponent_goal)
  columns2.extend(away_team_history_opponent_goal)
  if mode=="remove":
    for col in columns:
      input_df=input_df.filter(input_df[col]<=11)
    for col in columns2:
      input_df=input_df.filter(input_df[col]<=5)
  else:
    for col in columns:
      input_df = input_df.withColumn(col, when(input_df[col].cast(IntegerType()) >= 11,str(11.0)).otherwise(input_df[col]))
    for col in columns2:
      input_df = input_df.withColumn(col, when(input_df[col].cast(IntegerType()) >= 5,str(5.0)).otherwise(input_df[col]))
      
  return input_df

# Feature Engineering - Using MapReduce

## Calculating Columns Differences in Map function

In [ ]:
def Diff_map(data):
    result=(data[header.index("id")],{})
    for i in range(1,11):
      result[1]['home_team_history_goal_difference_'+str(i)] = float(data[header.index('home_team_history_goal_'+str(i))])-float(data[header.index('home_team_history_opponent_goal_'+str(i))])
      result[1]['away_team_history_goal_difference_'+str(i)] = float(data[header.index('away_team_history_goal_'+str(i))])-float(data[header.index('away_team_history_opponent_goal_'+str(i))])
      result[1]['home_team_history_rating_difference_'+str(i)] = float(data[header.index('home_team_history_rating_'+str(i))])-float(data[header.index('home_team_history_opponent_rating_'+str(i))])
      result[1]['away_team_history_rating_difference_'+str(i)] = float(data[header.index('away_team_history_rating_'+str(i))])-float(data[header.index('away_team_history_opponent_rating_'+str(i))])
      result[1]['home_team_history_date_difference_'+str(i)] = abs(int((dt.strptime(data[header.index('match_date')].split(' ')[0], "%Y-%m-%d") - dt.strptime(data[header.index('home_team_history_match_date_'+str(i))].split(' ')[0], "%Y-%m-%d")).days))  
      result[1]['away_team_history_date_difference_'+str(i)] = abs(int((dt.strptime(data[header.index('match_date')].split(' ')[0], "%Y-%m-%d") - dt.strptime(data[header.index('away_team_history_match_date_'+str(i))].split(' ')[0], "%Y-%m-%d")).days))  
    
    return result

## Returning Columns Differences in Reduce function

In [ ]:
def diff_reduce(x,y):
  columns=y[1]
  temp_dict={}
  for i in range(1,11):
      temp_dict['home_team_history_goal_difference_'+str(i)] = columns['home_team_history_goal_difference_'+str(i)]
      temp_dict['away_team_history_goal_difference_'+str(i)] = columns['away_team_history_goal_difference_'+str(i)]
      temp_dict['home_team_history_rating_difference_'+str(i)] = columns['home_team_history_rating_difference_'+str(i)]
      temp_dict['away_team_history_rating_difference_'+str(i)] = columns['away_team_history_rating_difference_'+str(i)]
      temp_dict['home_team_history_date_difference_'+str(i)] = columns['home_team_history_date_difference_'+str(i)]
      temp_dict['away_team_history_date_difference_'+str(i)] = columns['away_team_history_date_difference_'+str(i)]
  diff_columns[y[0]]=temp_dict
           
  return diff_columns 

## Calculating Columns Avg in Reduce function

In [ ]:
def avg_reduce(x,y):
  columns=y[1]

  home_team_history_goal_avg = round((float(columns["home_team_history_goal_difference_1"])+float(columns["home_team_history_goal_difference_2"])
  +float(columns["home_team_history_goal_difference_3"])+float(columns["home_team_history_goal_difference_4"])+float(columns["home_team_history_goal_difference_5"])
  +float(columns["home_team_history_goal_difference_6"])+float(columns["home_team_history_goal_difference_7"])+float(columns["home_team_history_goal_difference_8"])
  +float(columns["home_team_history_goal_difference_9"])+float(columns["home_team_history_goal_difference_10"]))/10)
  
  away_team_history_goal_avg = round((float(columns["away_team_history_goal_difference_1"])+float(columns["away_team_history_goal_difference_2"])
  +float(columns["away_team_history_goal_difference_3"])+float(columns["away_team_history_goal_difference_4"])+float(columns["away_team_history_goal_difference_5"])
  +float(columns["away_team_history_goal_difference_6"])+float(columns["away_team_history_goal_difference_7"])+float(columns["away_team_history_goal_difference_8"])
  +float(columns["away_team_history_goal_difference_9"])+float(columns["away_team_history_goal_difference_10"]))/10)

  home_team_history_rating_avg = (float(columns["home_team_history_rating_difference_1"])+float(columns["home_team_history_rating_difference_2"])
  +float(columns["home_team_history_rating_difference_3"])+float(columns["home_team_history_rating_difference_4"])+float(columns["home_team_history_rating_difference_5"])
  +float(columns["home_team_history_rating_difference_6"])+float(columns["home_team_history_rating_difference_7"])+float(columns["home_team_history_rating_difference_8"])
  +float(columns["home_team_history_rating_difference_9"])+float(columns["home_team_history_rating_difference_10"]))/10

  away_team_history_rating_avg = (float(columns["away_team_history_rating_difference_1"])+float(columns["away_team_history_rating_difference_2"])
  +float(columns["away_team_history_rating_difference_3"])+float(columns["away_team_history_rating_difference_4"])+float(columns["away_team_history_rating_difference_5"])
  +float(columns["away_team_history_rating_difference_6"])+float(columns["away_team_history_rating_difference_7"])+float(columns["away_team_history_rating_difference_8"])
  +float(columns["away_team_history_rating_difference_9"])+float(columns["away_team_history_rating_difference_10"]))/10
  avg_columns[y[0]]={
          "home_team_history_goal_avg":home_team_history_goal_avg,
           "away_team_history_goal_avg":away_team_history_goal_avg,
           "home_team_history_rating_avg":home_team_history_rating_avg,
           "away_team_history_rating_avg":away_team_history_rating_avg
           }
  
  
  return avg_columns

## Adding the New Features

In [ ]:

def add_features(output_df,filePath='train_preprocessed.csv'):

  avg_result=[]
  diff_result=[]
  global header 
  with open(filePath, 'r', encoding="utf8") as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader) 
    if header != None: 
      avg_result=functools.reduce(avg_reduce, map(Diff_map, csv_reader))

  with open(filePath, 'r', encoding="utf8") as read_obj:
    csv_reader = reader(read_obj)
    
    header = next(csv_reader)
    if header != None: 
      diff_result=functools.reduce(diff_reduce, map(Diff_map, csv_reader))
     
  diff_results_df=pd.DataFrame.from_dict(diff_result,orient='index')
  diff_results_df['id'] = diff_results_df.index
  diff_results_df['id']=diff_results_df['id'].astype(int)

  avg_results_df=pd.DataFrame.from_dict(avg_result,orient='index')
  avg_results_df['id'] = avg_results_df.index
  avg_results_df['id']=avg_results_df['id'].astype(int)

  output_df= pd.merge(left=output_df, right=diff_results_df, how='outer', on="id")
  output_df= pd.merge(left=output_df, right=avg_results_df, how='outer', on="id")
  output_df.dropna(axis=0,how="any",inplace=True)

  return output_df


# Normalization

In [ ]:
def normalize_columns(input_df, train = True):
  output_df = input_df.copy()

  normalized_columns = list()

  for column in output_df.columns:
    if column in normalized_columns:
      output_df[column] = (output_df[column] - output_df[column].min()) / (output_df[column].max() - output_df[column].min())	

  return output_df

In [ ]:
def normalize_spark_columns(input_df, train = True):
  output_df = input_df.copy()

  normalized_columns = list()
  # New Features Columns
  normalized_columns=["home_team_history_goal_avg","away_team_history_goal_avg","home_team_history_rating_avg","away_team_history_rating_avg"]
  for i in range(1,11):
      normalized_columns.extend(['home_team_history_goal_difference_'+str(i)])
      normalized_columns.extend(['away_team_history_goal_difference_'+str(i)])
      normalized_columns.extend(['home_team_history_rating_difference_'+str(i)])
      normalized_columns.extend(['away_team_history_rating_difference_'+str(i)])
  # History Rating
  normalized_columns.extend(home_team_history_rating)
  normalized_columns.extend(away_team_history_rating)

  # History Opponent Rating
  normalized_columns.extend(home_team_history_opponent_rating)
  normalized_columns.extend(away_team_history_opponent_rating)

  # History Goal
  normalized_columns.extend(home_team_history_goal)
  normalized_columns.extend(away_team_history_goal)

  # History Opponent Goal
  normalized_columns.extend(home_team_history_opponent_goal)
  normalized_columns.extend(away_team_history_opponent_goal)

  
  for column in output_df.columns:
    if column in normalized_columns:
      output_df[column] = (output_df[column] - output_df[column].min()) / (output_df[column].max() - output_df[column].min())	

  return output_df

# Encoding

In [ ]:
def categorical_encoding(input_df, train = True):
  output_df = input_df.copy()

  categorical_columns = []

  if train:
    target_enc = pd.get_dummies(output_df.target)
    output_df = output_df.join(target_enc)

    le = LabelEncoder()
    label = le.fit_transform(output_df['target'])
    output_df['target'] = label
  else:
    pass  

  for column in output_df.columns:
    if column in categorical_columns:
      current_column = pd.get_dummies(output_df[column], prefix=column)
      output_df = output_df.join(current_column)


  return output_df  

In [ ]:
def categorical_spark_encoding(input_df, train = True):
  output_df = input_df.copy()
  categorical_columns=list()
  categorical_columns=["is_cup"]
  
  if train:
    le = LabelEncoder()
    label = le.fit_transform(output_df['target'])
    output_df['target'] = label

  categorical_columns.extend(home_team_history_is_play_home)
  categorical_columns.extend(home_team_history_is_cup)
  #categorical_columns.extend(home_team_history_coach)
  categorical_columns.extend(away_team_history_is_play_home)
  categorical_columns.extend(away_team_history_is_cup)
  #categorical_columns.extend(away_team_history_coach)


  encoder = ce.BinaryEncoder(cols=categorical_columns)
  output_df = encoder.fit_transform(output_df)

  return output_df  

# Drop Columns

In [ ]:
def drop_columns(input_df, train = True):

  output_df = input_df

  columns = list()
  columns = ['home_team_name', 'away_team_name', 'league_id', 'league_name', 'match_date', 'home_team_coach_id', 'away_team_coach_id']  

  # History Match Date
  columns.extend(home_team_history_match_date)
  columns.extend(away_team_history_match_date)

  # History Coach
  columns.extend(home_team_history_coach)
  columns.extend(away_team_history_coach)

  # History League ID
  columns.extend(home_team_history_league_id)
  columns.extend(away_team_history_league_id)

  # History is Play Home
  # columns.extend(home_team_history_is_play_home)
  # columns.extend(away_team_history_is_play_home)

  # History Rating
  # columns.extend(home_team_history_rating)
  # columns.extend(away_team_history_rating)

  # History Opponent Rating
  # columns.extend(home_team_history_opponent_rating)
  # columns.extend(away_team_history_opponent_rating)

  # History Goal
  # columns.extend(home_team_history_goal)
  # columns.extend(away_team_history_goal)

  # History Opponent Goal
  # columns.extend(home_team_history_opponent_goal)
  # columns.extend(away_team_history_opponent_goal)

  # History is Cup
  # columns.extend(home_team_history_is_cup)
  # columns.extend(away_team_history_is_cup)


  for column in columns:
    output_df.drop(column, axis=1, inplace=True)
  # output_df=input_df.drop(*columns)
 
  return output_df

# Pipeline

In [ ]:
def pipeline(input_df, train = True):
  output_df = input_df.copy()
  # Removing null values and NANs
  output_df = remove_NAN(output_df, train)
  # Changing the type from String to Date in Date columns
  #output_df = handle_dates(output_df)


  # Feature Engineereing
  output_df = add_features(output_df)
  # print(output_df['match_date_time'])

  # Normalization
  output_df =  normalize_columns(output_df)

  # Remove Outlier (Numerical Data)
  output_df = remove_outliers(output_df)

  # Encoding (Categorical Data)
  output_df = categorical_encoding(output_df, train)

  # Last step: Drop Irrelevant Columns
  output_df = drop_columns(output_df, train)
 

  return output_df

In [ ]:
def spark_pipeline(input_df, train = True, mode="MaxAvg"):
  output_df = input_df
  # Removing null values and NANs
  output_df=remove_NAN(output_df, train, mode) 
  # Remove Outlier (Numerical Data)
  output_df = remove_outliers_spark(output_df,mode="fill") 

  # Intermediate Step for map reduce
  output_pandas = output_df.toPandas()
  output_pandas.to_csv('train_preprocessed.csv')

  # Feature Engineereing
  df_processed=pd.read_csv('train_preprocessed.csv')  
  df_processed['id']=df_processed['id'].astype(int)
  output_df=add_features(df_processed,filePath='train_preprocessed.csv')
  
  # Normalization
  output_df =  normalize_spark_columns(output_df)

  # Encoding (Categorical Data)
  output_df = categorical_spark_encoding(output_df, train) #468 columns

  # Last step: Drop Irrelevant Columns
  output_df = drop_columns(output_df, train)
  output_df.drop("Unnamed: 0", axis=1, inplace=True)

  return output_df

# Main

## Train

In [ ]:
train_spark_df = spark_pipeline(matches_spark_df)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [ ]:
print(len(train_spark_df))
train_spark_df.to_pickle("train.pkl")

110937


## Test

In [ ]:
test_spark_df = spark_pipeline(matches_test_spark_df, train = False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [ ]:
test_spark_df.to_pickle("test.pkl")

In [ ]:
test_spark_df_final = test_spark_df.drop(['id'], axis = 1)
test_spark_df_final

,is_cup_0,is_cup_1,home_team_history_is_play_home_1_0,home_team_history_is_play_home_1_1,home_team_history_is_play_home_2_0,home_team_history_is_play_home_2_1,home_team_history_is_play_home_3_0,home_team_history_is_play_home_3_1,home_team_history_is_play_home_4_0,home_team_history_is_play_home_4_1,...,home_team_history_goal_difference_10,away_team_history_goal_difference_10,home_team_history_rating_difference_10,away_team_history_rating_difference_10,home_team_history_date_difference_10,away_team_history_date_difference_10,home_team_history_goal_avg,away_team_history_goal_avg,home_team_history_rating_avg,away_team_history_rating_avg
1,0,1,0,1,0,1,0,1,0,1,...,0.5,0.3,0.425055,0.477955,70.0,69.0,0.375,0.500,0.427344,0.520775
2,0,1,1,0,1,0,0,1,0,1,...,0.7,0.4,0.414903,0.486332,248.0,169.0,0.375,0.375,0.366335,0.412096
3,0,1,0,1,1,0,0,1,0,1,...,0.6,0.3,0.292652,0.479462,249.0,139.0,0.375,0.500,0.370348,0.655452
4,0,1,0,1,1,0,0,1,0,1,...,0.5,0.4,0.482588,0.356314,235.0,239.0,0.500,0.500,0.448991,0.363205
5,0,1,0,1,1,0,0,1,0,1,...,0.3,0.5,0.530251,0.524249,146.0,154.0,0.375,0.500,0.487157,0.415115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72706,0,1,1,0,1,0,0,1,0,1,...,0.3,0.7,0.403282,0.597483,68.0,68.0,0.500,0.625,0.565661,0.634179
72707,0,1,0,1,0,1,1,0,1,0,...,0.4,0.4,0.380944,0.643581,58.0,64.0,0.625,0.500,0.612283,0.499575
72708,0,1,1,0,0,1,1,0,0,1,...,0.5,0.5,0.349382,0.828419,629.0,172.0,0.375,0.625,0.405156,0.587544
72709,0,1,0,1,1,0,1,0,1,0,...,0.6,0.4,0.579706,0.495976,31.0,41.0,0.625,0.625,0.703155,0.503874


# Training

In [ ]:
y_spark = train_spark_df[['target']]
x_spark = train_spark_df.drop(['id','target'],axis=1)

## Splitting Train & Validation

In [ ]:
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(x_spark, y_spark, test_size = 0.1, random_state = 150)

## Model

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
#!pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
# clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# clf.fit(X_train, y_train.values.ravel())
          # 0 -> Away
         # 2 -> Home
         # 1 -> Draw

In [ ]:
#pipe = make_pipeline(StandardScaler(), LogisticRegression(multi_class='multinomial', solver='lbfgs'))
#pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier())
#pipe=KNeighborsClassifier(3) #0.39976563908418966
#pipe=RandomForestClassifier() #0.4821525148729043
#pipe=AdaBoostClassifier() #0.48954389760230754
pipe=CatBoostClassifier() #0.49098611862267894
#pipe=LogisticRegression(multi_class='multinomial', solver='lbfgs') #0.49531278168379306

pipe.fit(X_train, y_train)


In [ ]:
score = pipe.score(X_train, y_train)
print("Score:", score)

In [ ]:
yhat = pipe.predict(X_test)
yhat_prob=pipe.predict_proba(X_test)
print(np.unique(yhat))
print(np.unique(y_test))

[0 1 2]
[0 1 2]


In [ ]:
#mean_squared_error(yhat, y_test)
print(log_loss(y_test, yhat_prob))

In [ ]:
print(classification_report(y_test, yhat))


              precision    recall  f1-score   support

           0       0.46      0.46      0.46      3484
           1       0.38      0.01      0.03      2779
           2       0.51      0.80      0.62      4831

    accuracy                           0.50     11094
   macro avg       0.45      0.42      0.37     11094
weighted avg       0.46      0.50      0.42     11094



In [ ]:
pipe.predict_proba(X_test)

array([[0.63589437, 0.18741244, 0.1766932 ],
       [0.20537493, 0.24554037, 0.5490847 ],
       [0.28812593, 0.34574553, 0.36612855],
       ...,
       [0.18028123, 0.20372789, 0.61599088],
       [0.17314631, 0.22804022, 0.59881347],
       [0.40378542, 0.28695238, 0.3092622 ]])

# Predict

In [ ]:
pred_arr = pipe.predict_proba(test_spark_df_final)

In [ ]:
pred_arr

array([[0.3365092 , 0.28523346, 0.37825734],
       [0.32790716, 0.33867813, 0.33341472],
       [0.29361436, 0.34623758, 0.36014806],
       ...,
       [0.47985072, 0.30923155, 0.21091772],
       [0.23362975, 0.29336842, 0.47300183],
       [0.3039839 , 0.32408694, 0.37192915]])

In [ ]:
pipe.classes_

array([0, 1, 2])

In [ ]:
final_df = pd.DataFrame({'id': test_spark_df['id']})

In [ ]:
final_df['home'] = pred_arr[:,2]

In [ ]:
final_df['draw'] = pred_arr[:,1]

In [ ]:
final_df['away'] = pred_arr[:,0]

In [ ]:
final_df

,id,home,draw,away
1,17695487,0.333243,0.333298,0.333459
2,17715496,0.333618,0.332380,0.334002
3,17715493,0.330481,0.333083,0.336436
4,17715492,0.335130,0.332713,0.332156
5,17715490,0.333861,0.332664,0.333475
...,...,...,...,...
72706,18450246,0.334089,0.332803,0.333108
72707,18164889,0.337977,0.332926,0.329097
72708,18449018,0.329948,0.332447,0.337605
72709,17958831,0.338636,0.332204,0.329159


In [ ]:
final_df.set_index('id', inplace=True)

In [ ]:
final_df.to_csv('catboost.csv')

# Save Weights

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/train.pkl" '/content/gdrive/MyDrive/Big Data'
!cp -r "/content/test.pkl" '/content/gdrive/MyDrive/Big Data'

In [ ]:
!cp -r "/content/sub_5th.csv" '/content/gdrive/MyDrive/Big Data'

# Debug

In [ ]:
train_spark_df # 0 -> Away
         # 2 -> Home
         # 1 -> Draw

In [ ]:
train_list = list()
test_list = list()
for col in train_spark_df.columns:
  train_list.append(col)
  #print(col)
for col in test_spark_df.columns:
  test_list.append(col)
  #print(col)

for item in train_list:
  if item not in test_list:
    print(item)
# +3 match_date, home-away coach id
# +10 history match date home
# +10 history match date away 
# +20 home away history coach
# -1 match date hour
# -20 home history match date hour
# +4 average
# +2 date difference
# -20 date difference
# total +8

target
home_team_coach_id_13
away_team_coach_id_13
home_team_history_coach_1_13
home_team_history_coach_2_13
home_team_history_coach_3_13
home_team_history_coach_4_13
home_team_history_coach_5_13
home_team_history_coach_6_13
home_team_history_coach_7_13
away_team_history_coach_3_13
away_team_history_coach_4_13
away_team_history_coach_5_13
away_team_history_coach_6_13
away_team_history_coach_7_13


In [ ]:
train_df.describe(include='all')

In [ ]:
for column in train_df.columns:
  print(column)

In [ ]:
train_df[['home_team_history_rating_1', 'home_team_history_opponent_rating_1', 'home_team_history_rating_difference_1']]

In [ ]:
train_df[['home_team_history_goal_1', 'home_team_history_opponent_goal_1', 'home_team_history_goal_difference_1']]

In [ ]:
# train_df[['match_date', 'home_team_history_match_date_10', 'home_team_history_date_difference_10']]